In [24]:
# !pip install pulp

In [25]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [26]:
data_path='data.csv'
data=pd.read_csv(data_path,encoding='cp949')
data.head()

,수요지 id,X,Y,초등학령인구
0,C1,126.371505,37.463782,1
1,C2,126.479505,37.495782,46
2,C3,126.481505,37.499782,36
3,C4,126.482505,37.495782,81
4,C5,126.483505,37.497782,25


In [27]:
target_path='target.csv'
target=pd.read_csv(target_path,encoding='cp949')
target.head()

,학교 id,X,Y
0,F1,126.543921,37.467292
1,F2,126.568726,37.496302
2,F3,126.492770,37.484428
3,F4,126.629683,37.495781
4,F5,126.528286,37.450285


In [28]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (킬로미터)

    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 위도와 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 허버사인 공식
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c

    return distance

In [29]:
s=data['초등학령인구'].sum()
s/750

13.614666666666666

In [30]:
s/200

51.055

In [31]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 19
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------
g = [1]*50
for j in [0, 1, 2, 4, 28, 32, 33, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49]:
    g[j] = 3 
# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] * g[j] for i in range(num_i) for j in range(num_j)])

# 제약조건 설정 부분
# ----------------------------------------------
# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(num_i):
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 그 값이 200과 750 사이에 있도록 제약 조건을 설정합니다.
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j >= 200 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# 각 거리 b_ij는 최대 1.5km까지만 허용됩니다.
for i in range(num_i):
    # 가능한 시설물 할당 리스트 구하기
    possible_assignments = [j for j in range(num_j) if b[i][j] <= 2]
    if b[i][j] > 2 :
        prob += l[i][j] == 0 
    

# 시설물 입지점 1, 4, 5, 7, 11, 15 가 할당되지 않도록 하는 제약식 추가
excluded_facility_indices = [0, 3, 4, 6, 10, 18]  # 시설물 입지점 인덱스는 0부터 시작

for j in excluded_facility_indices:
    prob += k[j] == 0
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {j+1}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {j+1}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': j+1, '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': j+1, '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)

Status: Optimal
수요지 1는 시설물 입지점 39에 할당됨
수요지 2는 시설물 입지점 45에 할당됨
수요지 3는 시설물 입지점 47에 할당됨
수요지 4는 시설물 입지점 45에 할당됨
수요지 5는 시설물 입지점 45에 할당됨
수요지 6는 시설물 입지점 47에 할당됨
수요지 7는 시설물 입지점 47에 할당됨
수요지 8는 시설물 입지점 47에 할당됨
수요지 9는 시설물 입지점 47에 할당됨
수요지 10는 시설물 입지점 45에 할당됨
수요지 11는 시설물 입지점 47에 할당됨
수요지 12는 시설물 입지점 47에 할당됨
수요지 13는 시설물 입지점 3에 할당됨
수요지 14는 시설물 입지점 45에 할당됨
수요지 15는 시설물 입지점 45에 할당됨
수요지 16는 시설물 입지점 45에 할당됨
수요지 17는 시설물 입지점 47에 할당됨
수요지 18는 시설물 입지점 47에 할당됨
수요지 19는 시설물 입지점 47에 할당됨
수요지 20는 시설물 입지점 47에 할당됨
수요지 21는 시설물 입지점 47에 할당됨
수요지 22는 시설물 입지점 47에 할당됨
수요지 23는 시설물 입지점 3에 할당됨
수요지 24는 시설물 입지점 45에 할당됨
수요지 25는 시설물 입지점 44에 할당됨
수요지 26는 시설물 입지점 3에 할당됨
수요지 27는 시설물 입지점 44에 할당됨
수요지 28는 시설물 입지점 3에 할당됨
수요지 29는 시설물 입지점 44에 할당됨
수요지 30는 시설물 입지점 44에 할당됨
수요지 31는 시설물 입지점 47에 할당됨
수요지 32는 시설물 입지점 47에 할당됨
수요지 33는 시설물 입지점 40에 할당됨
수요지 34는 시설물 입지점 47에 할당됨
수요지 35는 시설물 입지점 40에 할당됨
수요지 36는 시설물 입지점 44에 할당됨
수요지 37는 시설물 입지점 47에 할당됨
수요지 38는 시설물 입지점 39에 할당됨
수요지 39는 시설물 입지점 40에 할당됨
수요지 40는 시설물 입지점 39에 할당됨
수요지 41는 시설물 입지점 39에 할당됨
수요지 42는 시설물 입